***

##  **pandas**

Pandas to biblioteka dostarczająca wysoko-poziomowe struktury danych, przede wszystkim:

* **Series** (jednowymiarowa struktura, coś jak tablica lub kolumna danych z indeksem),

* **DataFrame** (dwuwymiarowa tabela danych z nazwanymi kolumnami i indeksowanymi wierszami).

Pandas jest zbudowane na NumPy, ale oferuje wygodne narzędzia do manipulacji danymi tabelarycznymi (podobnie jak arkusze kalkulacyjne czy baza danych). W DataFrame każda kolumna może mieć inny typ danych (np. jedna kolumna liczby całkowite, inna tekst, inna daty), co odróżnia je od sztywnych tablic NumPy, gdzie cały zestaw jest jednego typu.

W tej części powtórzymy najważniejsze operacje w pandas: tworzenie struktur, wczytywanie danych, wybieranie podzbiorów, filtrowanie, transformacje kolumn, grupowanie, łączenie, reorganizacja tabel (pivot/melt), uzupełnianie braków oraz pułapki typu *SettingWithCopyWarning*.

Na wstępie zaimportujmy pandas i ewentualnie numpy (często używane razem):

In [1]:
import pandas as pd
import numpy as np  # np może przydać się w niektórych operacjach




### **Series i DataFrame – tworzenie i podstawowa struktura**

**Series** to jedno-wymiarowy wektor wartości z przypisanym indeksem. Można go utworzyć np. z listy lub numpy array:


In [3]:
np.array([10, 20, 30, 40])

array([10, 20, 30, 40])

In [2]:
s = pd.Series([10, 20, 30, 40])
print(s)


0    10
1    20
2    30
3    40
dtype: int64


In [4]:
type(s)

pandas.core.series.Series

In [5]:
s.values

array([10, 20, 30, 40])

In [6]:
type(s.values)

numpy.ndarray

In [7]:
s.index

RangeIndex(start=0, stop=4, step=1)


Series został automatycznie zindeksowany od 0 do 3 (tak jak lista). Po lewej widzimy indeksy, po prawej wartości. **`dtype: int64`** na dole informuje o typie danych. Możemy zauważyć, że Series zachowuje się podobnie do 1D tablicy, ale posiada także **indeks** (który może być np. etykietą tekstową, datą itp., niekoniecznie liczbą).

Możemy też samemu podać indeks podczas tworzenia Series:

In [8]:

s2 = pd.Series([15, 25, 35], index=["a", "b", "c"])
print(s2)
print("Wartość o indeksie 'b':", s2["b"])


a    15
b    25
c    35
dtype: int64
Wartość o indeksie 'b': 25


In [15]:
# create a Series from dictionary
d = {"a": 1, "b": 2, "c": 3}
s3 = pd.Series(d)
print(s3)

a    1
b    2
c    3
dtype: int64


Jak widać, indeksami **`s2`** są teraz znaki 'a', 'b', 'c'. Odwołujemy się do wartości poprzez **`s2["b"]`**. Można też nadal używać indeksowania liczbowego (np. **`s2[1]`** da 25, czyli "drugi" element).




**DataFrame** to dwuwymiarowa struktura (w uproszczeniu zbiór Series ułożonych kolumnowo, dzielących ten sam indeks wierszy). DataFrame można utworzyć na różne sposoby. Najczęściej:

* z **słownika** gdzie kluczami są nazwy kolumn, a wartościami listy/serie z danymi,

* z listy słowników (gdzie każdy słownik to wiersz),

* z tablicy NumPy,

* z pliku (CSV, Excel, SQL, JSON, itp.).

Utwórzmy prosty DataFrame z słownika Python:

In [25]:


data = {
    "country": ["Poland", "USA", "UK", "Poland"],
    "year": [2020, 2021, 2021, 2022],
    "gdp": [595.86, 21137.47, 3126.31, 660.54]  # przykładowe dane GDP (mld USD)
}
df = pd.DataFrame(data)
display(df)

# import json
# # Zapisz DataFrame do pliku JSON
# df.to_json("data.json", orient="records", lines=True)

,country,year,gdp
0,Poland,2020,595.86
1,USA,2021,21137.47
2,UK,2021,3126.31
3,Poland,2022,660.54


In [18]:
pd.read_json("data.json", orient="records", lines=True)

,country,year,gdp
0,Poland,2020,595.86
1,USA,2021,21137.47
2,UK,2021,3126.31
3,Poland,2022,660.54


Oczekiwany DataFrame:

* Indeksy wierszy zostaną automatycznie nadane (0,1,2,3).

* Kolumny to "country", "year", "gdp".

* Wartości z list trafiają pod odpowiednie kolumny.



Każda kolumna ma swój typ danych (pandas stara się dobrać optymalny typ, np. kolumna "year" będzie typu int64, "gdp" typu float64, "country" najpewniej typu object czyli ogólny typ dla tekstu). Możemy szybko sprawdzić informacje o DataFrame:


In [26]:

print(df.dtypes)   # typy danych w kolumnach
print(df.shape)    # rozmiar (liczba wierszy, liczba kolumn)
print(df.index)    # indeks wierszy
print(df.columns)  # nazwy kolumn

country     object
year         int64
gdp        float64
dtype: object
(4, 3)
RangeIndex(start=0, stop=4, step=1)
Index(['country', 'year', 'gdp'], dtype='object')




Ponadto, pomocne metody:

* **`df.head(n)`** – podejrzenie pierwszych n wierszy (domyślnie 5),

* **`df.tail(n)`** – ostatnie n wierszy,

* **`df.info()`** – zwięzłe info o DataFrame (liczba wierszy, kolumn, typy, ile nie-null),

* **`df.describe()`** – szybkie statystyki opisowe dla kolumn numerycznych (liczność, średnia, std, min, kwartyle, max).


In [29]:
df.head(10)

,country,year,gdp
0,Poland,2020,595.86
1,USA,2021,21137.47
2,UK,2021,3126.31
3,Poland,2022,660.54


In [30]:
print(df.head(2))
display(df.describe())
df.info()

  country  year       gdp
0  Poland  2020    595.86
1     USA  2021  21137.47


,year,gdp
count,4.000000,4.000000
mean,2021.000000,6380.045000
std,0.816497,9908.547112
min,2020.000000,595.860000
25%,2020.750000,644.370000
50%,2021.000000,1893.425000
75%,2021.250000,7629.100000
max,2022.000000,21137.470000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  4 non-null      object 
 1   year     4 non-null      int64  
 2   gdp      4 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 228.0+ bytes



*Uwaga:* **`df.info()`** wypisze informacje na standardowe wyjście (nie zwraca obiektu), więc w Jupyter zostanie wyświetlone bez **`print`**. Zawiera to m.in. ile jest niepustych wartości w każdej kolumnie i typ.




### **Tworzenie DataFrame z różnych źródeł (dict, listy, CSV)**

**Ze słownika** – jak pokazano wyżej. Długości list muszą się zgadzać (po jednej wartości na wiersz).

**Z listy słowników** – np.:


In [32]:

rows = [
    {"name": "Alice", "age": 25, "city": "New York"},
    {"name": "Bob", "age": 30, "city": "Paris"},
    {"name": "Charlie", "age": 35, "city": "London"}
]
df2 = pd.DataFrame(rows)
print(df2)

      name  age      city
0    Alice   25  New York
1      Bob   30     Paris
2  Charlie   35    London



Tutaj klucze słowników stały się kolumnami, a brakujące klucze w danym wierszu (gdyby jakiś wiersz miał mniej pól) byłyby uzupełnione wartością NaN.

**Z tablicy NumPy** – można przekazać **`np.array`** i podać nazwę kolumn:

In [35]:
arr = np.array([[1, 2, 3], [4, 5, 6]])
df_arr = pd.DataFrame(arr, columns=["A", "B", "C"])
print(df_arr)


   A  B  C
0  1  2  3
1  4  5  6



Jeśli nie podamy nazw kolumn, pandas przypisze domyślne numery 0,1,2,...

**Wczytanie z pliku CSV / innego źródła:**

Pandas potrafi wczytywać pliki CSV bardzo łatwo przy pomocy **`pd.read_csv("nazwa_pliku.csv")`**. Jeżeli plik posiada nagłówek z nazwami kolumn, pandas je wykorzysta. Można też wczytywać dane z URL, jeśli jest dostępny plik CSV online.

Dla spójności przykładu, skorzystamy z ogólnodostępnego zbioru danych. Pandas nie ma wbudowanych datasetów, ale biblioteka **seaborn** udostępnia kilka znanych datasetów. Jeśli mamy zainstalowany seaborn, można np. załadować "tips" (dane z restauracji: rachunki, napiwki itp.):


In [36]:
import seaborn as sns
tips = sns.load_dataset("tips")
print(tips.head())
tips.info()


   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


Jeżeli nie mamy seaborn, możemy spróbować pobrać CSV bezpośrednio z repozytorium:


In [37]:

url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv"
tips = pd.read_csv(url)


In [38]:
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [ ]:
tips.to_csv("tips.csv", index=False)  # Zapisz DataFrame do pliku CSV

In [ ]:
pd.read_csv("tips.csv", index_col=0, )  # Wczytaj DataFrame z pliku CSV, ustawiając kolumnę jako indeks

,tip,sex,smoker,day,time,size
total_bill,,,,,,
16.99,1.01,Female,No,Sun,Dinner,2
10.34,1.66,Male,No,Sun,Dinner,3
21.01,3.50,Male,No,Sun,Dinner,3
23.68,3.31,Male,No,Sun,Dinner,2
24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...
29.03,5.92,Male,No,Sat,Dinner,3
27.18,2.00,Female,Yes,Sat,Dinner,2
22.67,2.00,Male,Yes,Sat,Dinner,2


**`tips.head()`** pokaże 5 pierwszych wierszy, **`tips.info()`** [m.in](http://m.in). że dataset ma 244 wiersze, 7 kolumn (total\_bill, tip, sex, smoker, day, time, size), typy danych (kilka kolumn typu category może już być ustawione domyślnie).


In [41]:
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [47]:
# example df with custom index
df_custom_index = pd.DataFrame({
    "A": [1, 2, 3],
    "B": [4, 5, 6]
}, index=[3, 2, 1])

display(df_custom_index)

,A,B
3,1,4
2,2,5
1,3,6


In [54]:
df_custom_index['A'].loc[1]

np.int64(3)

### **Wybór danych: indeksowanie `.loc` vs `.iloc`**


W pandas mamy kilka sposobów indeksowania/wyszukiwania danych:




* **`df[column_name]`** – podstawowy sposób wybierania kolumny o nazwie **`column_name`** (zwraca Series).

* **`df.loc[...]`** – wybieranie na podstawie **etykiet** (label-based). Używamy etykiet wierszy i kolumn.

* **`df.iloc[...]`** – wybieranie na podstawie **indeksów liczbowych** (position-based), działające podobnie do numpy (0,1,2,...).

* **`df.at[label, column]` / `df.iat[i,j]`** – podobne do loc/iloc, ale do szybkiego dostępu pojedynczej wartości.




* Także możliwe: **maski boolean** (podobnie jak w NumPy) do filtrowania wierszy.

Zalecane jest korzystanie z **`.loc`** i **`.iloc`** dla czytelności i bezpieczeństwa (zwykły **`df[...]`** ma pewne złożone zasady, może np. tworzyć kopie lub wybierać wiersze jeśli podamy maskę, bywa źródłem SettingWithCopyWarning w pewnych sytuacjach).




**`df.loc[row_indexer, col_indexer]`:**

* **`row_indexer`** może być etykietą jednego wiersza, listą etykiet, wycinkiem (slice) po etykietach, maską boolean.

* **`col_indexer`** analogicznie dla kolumn (może być nazwa kolumny, lista nazw, slice po nazwach, maska).


* Jeśli chcemy wybrać wszystkie wiersze/kolumny w którymś wymiarze, używamy **`:`**.
Przykłady:


In [55]:
# Pokażmy przykład na DataFrame tips, np. 5 pierwszych wierszy dla kontekstu
print(tips.head(5))


   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


In [56]:

# Wybieranie kolumny:
col_day = tips["day"]        # Series z dniami

In [57]:
col_day

0       Sun
1       Sun
2       Sun
3       Sun
4       Sun
       ... 
239     Sat
240     Sat
241     Sat
242     Sat
243    Thur
Name: day, Length: 244, dtype: object

In [58]:
col_total = tips.total_bill  # alternatywnie poprzez atrybut (jeśli nazwa kolumny nie koliduje z metodą)

In [61]:

print(col_day.head())

0    Sun
1    Sun
2    Sun
3    Sun
4    Sun
Name: day, dtype: object


In [63]:


# Wybieranie wiersza o konkretnym indeksie (label):
print("Wiersz o indeksie 0 via loc:\\n", tips.loc[0])

# Wybieranie zakresu wierszy po etykietach:
print("Wiersze 0 do 3 via loc:\\n", tips.loc[0:3])  # UWAGA: loc INKLUZYWNY dla końca! Zwróci indeksy 0,1,2,3

Wiersz o indeksie 0 via loc:\n total_bill     16.99
tip             1.01
sex           Female
smoker            No
day              Sun
time          Dinner
size               2
Name: 0, dtype: object
Wiersze 0 do 3 via loc:\n    total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2


In [64]:
#timeseries df example
dates = pd.date_range("2023-01-01", periods=5, freq="D")
df_timeseries = pd.DataFrame({
    "value": [10, 20, 30, 40, 50]
}, index=dates)


In [67]:
df_timeseries.loc['2023-01-01':'2023-01-04']  # Wybieranie zakresu dat

,value
2023-01-01,10
2023-01-02,20
2023-01-03,30
2023-01-04,40


In [68]:


# To samo via iloc (po pozycji):
print("Wiersze 0 do 3 via iloc:\\n", tips.iloc[0:4])  # iloc działa jak Python slicing, koniec ekskluzywny (0,1,2,3)

# Wybieranie podzbioru wierszy i kolumn:
subset = tips.loc[0:3, ["total_bill", "tip", "day"]]
print("Podzbiór danych (wiersze 0-3, wybrane kolumny):\\n", subset)


Wiersze 0 do 3 via iloc:\n    total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
Podzbiór danych (wiersze 0-3, wybrane kolumny):\n    total_bill   tip  day
0       16.99  1.01  Sun
1       10.34  1.66  Sun
2       21.01  3.50  Sun
3       23.68  3.31  Sun




Zwróć uwagę na różnicę: **`loc[0:3]`** zwróci wiersze o etykietach od 0 do 3 **włącznie** (pandas indeks domyślny 0,1,2,... jest liczbowy, ale traktowany jako etykieta w **`loc`**). Natomiast **`iloc[0:4]`** zwróci wiersze 0..3, bo 4 jest wyłączone. Ta subtelność wynika z tego, że **`loc`** traktuje argumenty slice jako etykiety końcowe włącznie (inaczej niż slicing w Pythonie), aby np. daty jako etykiety też mogły być wybierane inclusive na końcu zakresu.





**Filtrowanie (maski boolean)**:

Często chcemy wybrać wiersze spełniające jakiś warunek. Tworzymy maskę (Series boolowską) i przekazujemy ją do **`df.loc[...]`** lub bezpośrednio do **`df[...]`**.

Przykład: z DataFrame **`tips`** wybierzemy tylko wiersze, gdzie kolumna **`day`** (dzień tygodnia) to "Sun" (niedziela):


In [79]:
tips.loc[tips['day']=='Sun',['total_bill', 'tip']]

,total_bill,tip
0,16.99,1.01
1,10.34,1.66
2,21.01,3.50
3,23.68,3.31
4,24.59,3.61
...,...,...
186,20.90,3.50
187,30.46,2.00
188,18.15,3.50
189,23.10,4.00


In [75]:
mask = tips["day"] == "Sun"
print(mask.head(5))   # przykładowe wartości maski dla pierwszych 5 wierszy

0    True
1    True
2    True
3    True
4    True
Name: day, dtype: bool


In [80]:

sun_tips = tips.loc[mask]    # wybieramy tylko wiersze gdzie mask == True
print(sun_tips.head())
print("Liczba wpisów dla niedzieli:", sun_tips.shape[0])


   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4
Liczba wpisów dla niedzieli: 76


In [85]:
sun_tips.iloc[0:50]

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2.00,Male,No,Sun,Dinner,2
7,26.88,3.12,Male,No,Sun,Dinner,4
8,15.04,1.96,Male,No,Sun,Dinner,2
9,14.78,3.23,Male,No,Sun,Dinner,2




Maska **`mask`** będzie Series zawierającą True/False dla każdego wiersza, gdzie True tam, gdzie **`day`** to "Sun". **`tips.loc[mask]`** zwróci DataFrame zawierający tylko wiersze z niedzieli. Alternatywnie mogliśmy zrobić **`tips[tips["day"] == "Sun"]`** – to często używana składnia skrócona (df\[mask]) dla filtrowania wierszy.

Możemy także łączyć warunki, np. wybrać tylko niedziele gdzie osoba paliła (**`smoker == "Yes"`**). Przy łączeniu warunków musimy użyć operatorów bitowych **`&`** (AND), **`|`** (OR), **`~`** (NOT) zamiast and/or, i każdy warunek brać w nawias:


In [ ]:
sun_smokers = tips[(tips["day"] == "Sun") & (tips["smoker"] == "Yes")]
print(sun_smokers.head())

     total_bill   tip     sex smoker  day    time  size
164       17.51  3.00  Female    Yes  Sun  Dinner     2
172        7.25  5.15    Male    Yes  Sun  Dinner     2
173       31.85  3.18    Male    Yes  Sun  Dinner     2
174       16.82  4.00    Male    Yes  Sun  Dinner     2
175       32.90  3.11    Male    Yes  Sun  Dinner     2


Powyższy przykład pokazuje filtrowanie: wiersze gdzie **`day`** to "Sun" **i** jednocześnie **`smoker`** to "Yes".



**Sortowanie danych**:

Pandas umożliwia sortowanie DataFrame po wartościach w kolumnach za pomocą **`df.sort_values()`**. Można podać jedną kolumnę lub listę kolumn oraz parametry:

* **`ascending=True/False`** dla rosnąco/malejąco,

* **`inplace=True`** aby posortować w miejscu (lub bez, aby zwrócić posortowaną kopię).

Przykład: Posortujmy **`tips`** malejąco po **`total_bill`**:

In [93]:
tips.sort_values("total_bill",ascending=False )

,total_bill,tip,sex,smoker,day,time,size
170,50.81,10.00,Male,Yes,Sat,Dinner,3
212,48.33,9.00,Male,No,Sat,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
156,48.17,5.00,Male,No,Sun,Dinner,6
182,45.35,3.50,Male,Yes,Sun,Dinner,3
...,...,...,...,...,...,...,...
149,7.51,2.00,Male,No,Thur,Lunch,2
111,7.25,1.00,Female,No,Sat,Dinner,1
172,7.25,5.15,Male,Yes,Sun,Dinner,2
92,5.75,1.00,Female,Yes,Fri,Dinner,2


In [95]:
tips_sorted = tips.sort_values("total_bill", ascending=False)
display(tips_sorted.head(5)[["total_bill", "tip", "day"]])

,total_bill,tip,day
170,50.81,10.00,Sat
212,48.33,9.00,Sat
59,48.27,6.73,Sat
156,48.17,5.00,Sun
182,45.35,3.50,Sun


To zwróci nowe DataFrame posortowane (oryginalny **`tips`** pozostaje niezmieniony, bo nie użyliśmy inplace). Jeżeli chcemy posortować według wielu kolumn (np. najpierw **`day`**, a w ramach tego **`total_bill`** rosnąco), możemy zrobić:


In [98]:
tips.sort_values(["day", "total_bill"], ascending=[False, True], inplace=True)
display(tips[["day", "total_bill", "tip"]])


,day,total_bill,tip
149,Thur,7.51,2.00
195,Thur,7.56,1.44
145,Thur,8.35,1.50
135,Thur,8.51,1.25
126,Thur,8.52,1.48
...,...,...,...
91,Fri,22.49,3.50
94,Fri,22.75,3.25
96,Fri,27.28,4.00
90,Fri,28.97,3.00




Tutaj sortujemy **`tips`** (inplace) najpierw po kolumnie "day" alfabetycznie, a dla wierszy z tym samym dniem – po wartości rachunku.




### **Dodawanie/usuwanie kolumn, operacje arytmetyczne na kolumnach**



**Dodawanie nowej kolumny:**

W pandas najprostszym sposobem jest przypisanie do nowej nazwy kolumny. Np. dodajmy kolumnę **`tip_pct`** do **`tips`**, która będzie procentem napiwku w stosunku do rachunku:


In [103]:
tips

,total_bill,tip,sex,smoker,day,time,size
149,7.51,2.00,Male,No,Thur,Lunch,2
195,7.56,1.44,Male,No,Thur,Lunch,2
145,8.35,1.50,Female,No,Thur,Lunch,2
135,8.51,1.25,Female,No,Thur,Lunch,2
126,8.52,1.48,Male,No,Thur,Lunch,2
...,...,...,...,...,...,...,...
91,22.49,3.50,Male,No,Fri,Dinner,2
94,22.75,3.25,Female,No,Fri,Dinner,2
96,27.28,4.00,Male,Yes,Fri,Dinner,2
90,28.97,3.00,Male,Yes,Fri,Dinner,2


In [ ]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"] * 100
print(tips.head(3)[["total_bill", "tip", "tip_pct"]])

     total_bill   tip    tip_pct
149        7.51  2.00  26.631158
195        7.56  1.44  19.047619
145        8.35  1.50  17.964072


In [107]:
tips.describe()['tip_pct']  # Statystyki dla kolumny tip_pct

count    244.000000
mean      16.080258
std        6.107220
min        3.563814
25%       12.912736
50%       15.476977
75%       19.147549
max       71.034483
Name: tip_pct, dtype: float64



Pandas policzy to wektorowo (dzielenie Series przez Series działa elementowo, podobnie mnożenie). **`tip_pct`** zostanie dodana do DataFrame.

Można też dodawać kolumny, których wartości zależą od warunku:

In [108]:
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pct
149,7.51,2.00,Male,No,Thur,Lunch,2,26.631158
195,7.56,1.44,Male,No,Thur,Lunch,2,19.047619
145,8.35,1.50,Female,No,Thur,Lunch,2,17.964072
135,8.51,1.25,Female,No,Thur,Lunch,2,14.688602
126,8.52,1.48,Male,No,Thur,Lunch,2,17.370892
...,...,...,...,...,...,...,...,...
91,22.49,3.50,Male,No,Fri,Dinner,2,15.562472
94,22.75,3.25,Female,No,Fri,Dinner,2,14.285714
96,27.28,4.00,Male,Yes,Fri,Dinner,2,14.662757
90,28.97,3.00,Male,Yes,Fri,Dinner,2,10.355540


In [109]:
tips["tip_pct"] > 20

149     True
195    False
145    False
135    False
126    False
       ...  
91     False
94     False
96     False
90     False
95     False
Name: tip_pct, Length: 244, dtype: bool

In [116]:
# Dodajmy kolumnę 'high_tip' = True/False, która oznaczy, czy tip_pct > 20%
tips["high_tip"] = tips["tip_pct"] > 20
print(tips.head(5)[["tip_pct", "high_tip"]])


       tip_pct  high_tip
149  26.631158      True
195  19.047619     False
145  17.964072     False
135  14.688602     False
126  17.370892     False


In [112]:
tips.describe(include='all')  # Statystyki dla wszystkich kolumn, w tym kategorycznych

,total_bill,tip,sex,smoker,day,time,size,tip_pct,high_tip
count,244.000000,244.000000,244,244,244,244,244.000000,244.000000,244
unique,NaN,NaN,2,2,4,2,NaN,NaN,2
top,NaN,NaN,Male,No,Sat,Dinner,NaN,NaN,False
freq,NaN,NaN,157,151,87,176,NaN,NaN,205
mean,19.785943,2.998279,NaN,NaN,NaN,NaN,2.569672,16.080258,NaN
std,8.902412,1.383638,NaN,NaN,NaN,NaN,0.951100,6.107220,NaN
min,3.070000,1.000000,NaN,NaN,NaN,NaN,1.000000,3.563814,NaN
25%,13.347500,2.000000,NaN,NaN,NaN,NaN,2.000000,12.912736,NaN
50%,17.795000,2.900000,NaN,NaN,NaN,NaN,2.000000,15.476977,NaN
75%,24.127500,3.562500,NaN,NaN,NaN,NaN,3.000000,19.147549,NaN



Tutaj wykorzystujemy maskę logiczną jako wartość – pandas automatycznie potraktuje True/False jako odpowiednik 1/0 czy bool w kolumnie.


Do usunięcia kolumny używa się metody **`df.drop`**. Należy podać nazwę kolumny i parametr **`axis=1`** (bo kolumny to oś 1), ewentualnie **`inplace=True`** jeśli chcemy modyfikować obecny DataFrame:


In [113]:
tips.drop(0)

,total_bill,tip,sex,smoker,day,time,size,tip_pct,high_tip
149,7.51,2.00,Male,No,Thur,Lunch,2,26.631158,True
195,7.56,1.44,Male,No,Thur,Lunch,2,19.047619,False
145,8.35,1.50,Female,No,Thur,Lunch,2,17.964072,False
135,8.51,1.25,Female,No,Thur,Lunch,2,14.688602,False
126,8.52,1.48,Male,No,Thur,Lunch,2,17.370892,False
...,...,...,...,...,...,...,...,...,...
91,22.49,3.50,Male,No,Fri,Dinner,2,15.562472,False
94,22.75,3.25,Female,No,Fri,Dinner,2,14.285714,False
96,27.28,4.00,Male,Yes,Fri,Dinner,2,14.662757,False
90,28.97,3.00,Male,Yes,Fri,Dinner,2,10.355540,False


In [117]:

tips.drop("high_tip", axis=1, inplace=True)  # usuwamy kolumnę high_tip, wynik przypisujemy z powrotem
# (alternatywnie tips.drop(..., inplace=True) bez przypisania)


In [118]:
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pct
149,7.51,2.00,Male,No,Thur,Lunch,2,26.631158
195,7.56,1.44,Male,No,Thur,Lunch,2,19.047619
145,8.35,1.50,Female,No,Thur,Lunch,2,17.964072
135,8.51,1.25,Female,No,Thur,Lunch,2,14.688602
126,8.52,1.48,Male,No,Thur,Lunch,2,17.370892
...,...,...,...,...,...,...,...,...
91,22.49,3.50,Male,No,Fri,Dinner,2,15.562472
94,22.75,3.25,Female,No,Fri,Dinner,2,14.285714
96,27.28,4.00,Male,Yes,Fri,Dinner,2,14.662757
90,28.97,3.00,Male,Yes,Fri,Dinner,2,10.355540



Można także usuwać wiele kolumn naraz: **`df.drop(["col1", "col2"], axis=1)`**.

Inny sposób: **`del df["col"]`** także usuwa kolumnę, jeśli nie potrzebujemy opcji inplace=False.



**Operacje na kolumnach (Series)**:

Kolumny DataFrame są Series, więc wszystkie operacje z NumPy/pandas na tablicach jednowymiarowych działają. Możemy np. znormalizować kolumnę, wykonując operacje arytmetyczne:


np.float64(19.78594262295082)

In [123]:
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pct
149,7.51,2.00,Male,No,Thur,Lunch,2,26.631158
195,7.56,1.44,Male,No,Thur,Lunch,2,19.047619
145,8.35,1.50,Female,No,Thur,Lunch,2,17.964072
135,8.51,1.25,Female,No,Thur,Lunch,2,14.688602
126,8.52,1.48,Male,No,Thur,Lunch,2,17.370892
...,...,...,...,...,...,...,...,...
91,22.49,3.50,Male,No,Fri,Dinner,2,15.562472
94,22.75,3.25,Female,No,Fri,Dinner,2,14.285714
96,27.28,4.00,Male,Yes,Fri,Dinner,2,14.662757
90,28.97,3.00,Male,Yes,Fri,Dinner,2,10.355540


In [124]:
max_bill = tips["total_bill"].max()
min_bill = tips["total_bill"].min()
tips["total_bill_range"] = (tips["total_bill"] - min_bill) / (max_bill - min_bill)
print(tips[["total_bill", "total_bill_range"]].head(5))


     total_bill  total_bill_range
149        7.51          0.093004
195        7.56          0.094051
145        8.35          0.110599
135        8.51          0.113951
126        8.52          0.114160


In [125]:
tips.describe()

,total_bill,tip,size,tip_pct,total_bill_range
count,244.000000,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672,16.080258,0.350145
std,8.902412,1.383638,0.951100,6.107220,0.186477
min,3.070000,1.000000,1.000000,3.563814,0.000000
25%,13.347500,2.000000,2.000000,12.912736,0.215281
50%,17.795000,2.900000,2.000000,15.476977,0.308442
75%,24.127500,3.562500,3.000000,19.147549,0.441087
max,50.810000,10.000000,6.000000,71.034483,1.000000



Dodaliśmy kolumnę **`total_bill_range`** gdzie wartości są przeskalowane do zakresu \[0,1] względem min i max rachunku.

Możemy też używać funkcji agregujących z **`df`** po osi:

* **`df.sum(axis=1)`** – da Series z sumą wartości w każdym wierszu (jeśli wszystkie są numeryczne lub np. zignoruje nienumeryczne),

* **`df.mean(axis=0)`** – średnia w każdej kolumnie (tylko numeryczne kolumny są brane domyślnie pod uwagę w takich operacjach).



### **Grupowanie i agregacja (`groupby`, `agg`)**

Bardzo mocną stroną pandas jest możliwość grupowania danych i agregowania wyników, podobnie do operacji "GROUP BY" w SQL. Używamy do tego metody **`df.groupby()`** zwracającej obiekt grupujący, na którym można wykonać agregacje (sumy, średnie itp.) dla każdej grupy.

**Przykład 1:** Policzmy średni **`tip`** (napiwek) dla każdej wartości kolumny **`day`** w zbiorze **`tips`** (średni napiwek w poszczególne dni tygodnia):


In [ ]:
grouped_by_day = tips.groupby("day")   # grupowanie wg kolumny 'day'
mean_tips_by_day = grouped_by_day["tip"].mean()
print(mean_tips_by_day)


Wynik **`mean_tips_by_day`** będzie Series: indeksami będą unikalne dni, a wartościami średni napiwek. Alternatywnie, można to zrobić w jednym łańcuchu:


In [ ]:
mean_tips_by_day = tips.groupby("day")["tip"].mean()





lub używając **`agg`**:


In [ ]:
mean_tips_by_day = tips.groupby("day").agg({"tip": "mean"})
print(mean_tips_by_day)

**`agg`** pozwala zdefiniować kilka agregacji naraz, nawet różne dla różnych kolumn.


**Przykład 2:** Policzmy liczbę obsług (wierszy) i średni rachunek dla każdego dnia tygodnia:


In [ ]:

agg_stats = tips.groupby("day").agg({
    "total_bill": "mean",
    "tip": "mean",
    "size": "count"
})
print(agg_stats)


To da DataFrame, gdzie indeksami są dni, a kolumny to 'total\_bill', 'tip', 'size' z zastosowaną odpowiednio średnią i count. Ponieważ **`size`** to count liczby wierszy, odpowiada ile było transakcji danego dnia.

Można również grupować po wielu kolumnach jednocześnie. Np. średni **`tip`** w podziale na dzień i porę dnia (**`time`** kolumna to Lunch/Dinner w **`tips`**):


In [ ]:


mean_tip_day_time = tips.groupby(["day", "time"])["tip"].mean()
print(mean_tip_day_time)


To utworzy grupy dla każdej kombinacji (day, time). Wynikiem będzie Series z MultiIndex lub można **`.reset_index()`** aby przekształcić to z powrotem do zwykłego DataFrame.

**Filtrowanie po grupowaniu:**

Czasem po agregacji chcemy zachować tylko grupy spełniające pewne kryterium (np. grupy z liczbą elementów > jakaś wartość). Do tego można użyć **`.filter`** na grupowaniu. Ale to bardziej zaawansowane, wspominam tylko.

Pandas posiada też metodę **`pivot_table`** która jednocześnie grupuje i unstackuje do formatu tabelarycznego (o tym w sekcji pivot).


### **Łączenie danych: merge / join**

Realistyczne scenariusze często mają dane w wielu tabelach, które trzeba ze sobą połączyć (po wspólnym kluczu). Pandas oferuje funkcję **`pd.merge()`** oraz metody **`DataFrame.merge()`** do łączenia DataFrame podobnie do operacji SQL join.

Podstawowe użycie:

```python

pd.merge(left_df, right_df, how="inner", on="key_column")


```





* **`left_df`**, **`right_df`** – DataFrames do połączenia,

* **`how`** – rodzaj złączenia: "inner", "left", "right", "outer" (domyślnie "inner"),

* **`on`** – nazwa kolumny (lub lista kolumn), po której łączymy (klucz). Jeśli w obu DataFrame kolumna nazywa się inaczej, używamy **`left_on`**, **`right_on`**.





**Przykład:** Załóżmy, że mamy dwa DataFrame:

* **`df_customers`** z kolumnami: **`customer_id`**, **`name`**, **`country`**

* **`df_orders`** z kolumnami: **`order_id`**, **`customer_id`**, **`amount`**


Chcemy połączyć informacje o zamówieniach z danymi klientów (dodać np. imię i kraj do każdej transakcji). Zakładamy, że **`customer_id`** jest kluczem.


In [ ]:

df_customers = pd.DataFrame({
    "customer_id": [101, 102, 103],
    "name": ["Alice", "Bob", "Charlie"],
    "country": ["USA", "USA", "UK"]
})
df_orders = pd.DataFrame({
    "order_id": [1001, 1002, 1003, 1004],
    "customer_id": [101, 103, 102, 101],
    "amount": [250, 500, 150, 300]
})
merged = pd.merge(df_orders, df_customers, how="inner", on="customer_id")
print(merged)


Jak widać, zamówienia zostały wzbogacone o dane klienta poprzez dopasowanie **`customer_id`**. Domyślnie **`merge`** robi "inner join", więc jeśli w którymś DataFrame nie było dopasowania, takie wiersze by odpadły. Możemy użyć **`how="left"`** aby zachować wszystkie wiersze lewego DataFrame (np. wszystkie orders nawet bez klienta) albo "outer" aby zachować wszystkie z obu stron (wypełniając NaN tam gdzie brak dopasowania).

Jeśli kolumna klucz nazywa się inaczej po dwóch stronach:


```python

pd.merge(df1, df2, left_on="left_id", right_on="right_id")


```



wtedy powstaną obie kolumny, ale **`suffixes=('_x','_y')`** może dokleić do powielających się kolumn.

**Metoda `.join`:** Alternatywnie, DataFrame ma metodę **`.join`** do łączenia wg indeksów, lub do łączenia kolumnowo DataFrame o tym samym indeksie. W praktyce **`.merge`** jest częściej używane bo jest bardziej elastyczne.



### **Pivot, melt – zmiana kształtu danych**

Czasami potrzebujemy zmienić organizację danych z formatu "długiego" (każdy pomiar w osobnym wierszu, z kolumną kategoria) na "szeroki" (osobne kolumny dla kategorii) lub odwrotnie. W pandas służą do tego:

* **`pivot`** / **`pivot_table`** – zmienia dane z długich na szerokie (tworzy nowe kolumny na podstawie wartości kolumn kategorii).

* **`melt`** – "roztapia" dane z formatu szerokiego do długiego (kolumny stają się wartościami w nowym wierszu).

**Przykład pivot:** Skorzystajmy z **`tips`**. Załóżmy, że chcemy stworzyć tabelę, gdzie indeksami będą dni (**`day`**), kolumny to pora dnia (**`time`**), a wartości to średnia wielkość rachunku (**`total_bill`**). Możemy użyć **`pivot_table`** (lepiej od pivot, bo od razu pozwala agregować):


In [ ]:

pivot = tips.pivot_table(values="total_bill", index="day", columns="time", aggfunc="mean")
print(pivot)




To stworzy tabelkę (DataFrame) gdzie wiersze = \['Thur','Fri','Sat','Sun'] (dni tygodnia obecne w tips), kolumny = \['Lunch','Dinner'], a wartości to średni rachunek. Brakujące kombinacje (np. może nie być Lunch w Sun? W tips dataset Fri Lunch albo Sun Lunch może nie występują) będą NaN. **`pivot_table`** radzi sobie z tym, **`pivot`** by wyrzucił błąd gdy kombinacje nie są unikalne bez agregacji.

**Przykład melt:** Odwróćmy powyższe – weźmy pivot i przywróćmy do "długiego" formatu:

In [ ]:
long = pivot.reset_index().melt(id_vars="day", value_vars=["Lunch","Dinner"], var_name="time", value_name="mean_bill")
print(long.head())




Tutaj:

* **`pivot.reset_index()`** przenosi **`day`** z indeksu do kolumny zwykłej, żeby **`melt`** mógł go traktować jak zwykłą kolumnę.

* **`melt`** z parametrami: **`id_vars="day"`** (ta kolumna ma zostać jako identyfikator), **`value_vars=["Lunch","Dinner"]`** (te kolumny przekształcamy z szerokich na wartości w wierszach), **`var_name="time"`** (nowa kolumna, dawny nagłówek kolumn), **`value_name="mean_bill"`** (nowa kolumna dla wartości).

* Rezultat to DataFrame z kolumnami: day, time, mean\_bill – i wierszami reprezentującymi kombinacje day-time z odpowiednią wartością średniego rachunku.

**Inny przykład pivot/melt:**

Jeśli mamy DataFrame:


```yaml

   country  year  value
0   Poland  2020    100
1   Poland  2021    150
2     USA   2020    200
3     USA   2021    210


```

I chcemy szeroki format z kolumnami year:

In [ ]:
df = pd.DataFrame({
    "country": ["Poland", "Poland", "USA", "USA"],
    "year": [2020, 2021, 2020, 2021],
    "value": [100, 150, 200, 210]
})
wide = df.pivot(index="country", columns="year", values="value")
print(wide)




I analogicznie **`.reset_index().melt()`** by powrócić do długiego formatu.



### **Obsługa brakujących danych (`isna`, `fillna`, `dropna`)**

W realnych danych często spotkamy **brakujące wartości**. Pandas reprezentuje brak danych jako **`NaN`** (not a number) dla danych numerycznych, lub **`None/NaN`** dla obiektowych. Dla typu **`float64`** NaN to specjalna wartość (typu float), dla typów całkowitych pandas używa specjalnego typu **`Int64`** (z dużym "I") aby obsłużyć NaN, lub zmienia na float64 domyślnie. Dla typu **`object`** może być None lub np. **`pd.NA`** (specjalna wartość "Not Available" w nowych wersjach pandasa, która działa w różnych typach, w tym kategorycznych, Int64 itp.).

Kilka podstawowych narzędzi:

* **`df.isna()`** – zwraca DataFrame/Series z wartościami True/False, gdzie True oznacza brak (NaN/NA).

* **`df.notna()`** – analogicznie, odwrotność.

* **`df.dropna()`** – usuwa wiersze (domyślnie) z brakami. Opcje: **`axis=1`** żeby usuwać kolumny z brakami, **`how="any"`** (usuwa wiersz jeśli *jakakolwiek* wartość NaN w wierszu), **`how="all"`** (usuwa tylko jeśli *wszystkie* wartości w wierszu NaN), **`subset=["col1","col2"]`** żeby ograniczyć do sprawdzania konkretnych kolumn.

* **`df.fillna(value)`** – uzupełnia brakujące wartości podaną stałą (albo np. **`method="ffill"`**/**`"bfill"`** do wypełnienia poprzednią/następną znaną wartością - forward/backward fill).


In [ ]:
df = pd.DataFrame({
    "A": [1, np.nan, 3, 4],
    "B": [np.nan, 2, np.nan, 5],
    "C": ["x", "y", None, "z"]
})
print(df)

Widzimy NaN (dla liczb) i None (dla C).

* Wykrywanie braków:


In [ ]:

print(df.isna())
print("Braki w kolumnie A:", df["A"].isna().sum())  # ile NaN w A


* Usuwanie braków:


In [ ]:
df_drop_rows = df.dropna()
print("Dropna (domyślnie, wiersze z ANY NaN):\\n", df_drop_rows)
df_drop_cols = df.dropna(axis=1)
print("Dropna kolumny (usuń kolumny z ANY NaN):\\n", df_drop_cols)
df_drop_all = df.dropna(how="all")
print("Dropna (usuwa wiersze z ALL NaN):\\n", df_drop_all)


* Uzupełnianie braków:


In [ ]:
df_filled_zero = df.fillna(0)
print("Braki uzupełnione zerem:\\n", df_filled_zero)

# Uzupełnijmy brakujące teksty w kolumnie C jakimś napisem, np. "brak"
df_filled = df.fillna({"A": 0, "B": df["B"].mean(), "C": "brak"})
print("Braki uzupełnione różnymi wartościami:\\n", df_filled)






W powyższym: dla kolumny A wypełniamy 0, dla B średnią z B (ignorując NaN przy liczeniu), dla C stringiem "brak". **`fillna`** akceptuje słownik kolumna->wartość.

**Interpolate:** W przypadku danych numerycznych czasem używa się **`df.interpolate()`** do wypełnienia braków przez interpolację (np. liniową) pomiędzy znanymi wartościami – przydaje się w szeregach czasowych.

**Uwaga:** W nowszych wersjach Pandas pojawia się typ **`pd.NA`** (podobny do R-owego NA) jako jednolite brakujące, który współpracuje z nowymi typami danymi (typy całkowite z brakami, typ boolean z brakami). Tutaj nie zagłębiamy się, ale warto wiedzieć, że praca z brakami staje się bardziej spójna.



### **SettingWithCopyWarning – poprawne przypisywanie danych**

Przechodzimy do zagadnienia, które bywa frustrujące dla wielu użytkowników pandas – ostrzeżenie **SettingWithCopyWarning**. To ostrzeżenie wyskakuje, gdy próbujemy zmodyfikować dane w DataFrame, ale pandas podejrzewa, że operujemy na **kopii** oryginalnych danych, a nie na samym oryginale. Dzieje się tak najczęściej przy używaniu tzw. *chain indexing*, czyli łączenia dwóch kroków indeksowania w jednym wyrażeniu.

**Przykład sytuacji generującej ostrzeżenie:**

In [99]:
df = pd.DataFrame({"A": [1, 2, 3, 4], "B": [10, 20, 30, 40]})

In [ ]:
sub = df[df["A"] > 2][]      # wybieramy wiersze gdzie A>2

In [101]:

sub["B"] = sub["B"] * 2    # próbujemy zmodyfikować kolumnę B w tym podzbiorze
print(sub)
print(df)


   A   B
2  3  60
3  4  80
   A   B
0  1  10
1  2  20
2  3  30
3  4  40


C:\Users\bart.bilski\AppData\Local\Temp\ipykernel_51344\1147765864.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["B"] = sub["B"] * 2    # próbujemy zmodyfikować kolumnę B w tym podzbiorze


In [102]:
df

,A,B
0,1,10
1,2,20
2,3,30
3,4,40


W momencie **`sub["B"] = ...`** pandas wyświetli ostrzeżenie:




Co się stało? **`df[df["A"] > 2]`** tworzy **nowy DataFrame** (kopia wybranych wierszy), przypisujemy go do **`sub`**. Następnie **`sub["B"] = sub["B"] * 2`** zmienia dane w **`sub`**, ale nie przenosi tych zmian do **`df`**. Pandas ostrzega, że operujemy na kopii danych źródłowych, więc jeśli intencją było zmienić oryginał, to tak się nie stanie.

Generalnie, SettingWithCopyWarning pojawia się zawsze, gdy pandas nie jest pewien, czy pracujesz na oryginalnym DataFrame czy na jego kopii (np. wyciągniętej przez **`[...]`**). Jest to ostrzeżenie (nie błąd), ale sygnalizuje, że potencjalnie Twoje przypisanie nie robi tego, co myślisz.



Jak poprawnie modyfikować dane, by uniknąć tego ostrzeżenia?

1. Użyć jednokrokowego indeksowania z **`.loc`** przy przypisaniu:

   * Zamiast robić **`sub = df[df["A"] > 2]; sub["B"] = ...`**, można bezpośrednio: **`df.loc[df["A"] > 2, "B"] = df.loc[df["A"] > 2, "B"] * 2`**. To modyfikuje kolumnę B dla wybranych wierszy oryginalnego **`df`** bez tworzenia pośredniej kopii.
2. Jeśli jednak potrzebujesz pracować na podzbiorze jako osobnym DataFrame (nie chcesz zmieniać oryginału), to utwórz kopię jawnie: **`sub = df[df["A"] > 2].copy()`**. Wtedy **`sub["B"] = ...`** zmieni tylko **`sub`** i nie będzie ostrzeżenia, bo jasno zaznaczyliśmy, że chcemy kopię.
3. Unikać konstrukcji typu **`df[col][mask] = ...`** – to klasyczny chain indexing (df\[col] daje Series, \[mask] filtruje – dwa kroki), lepiej **`df.loc[mask, col] = ...`**.

Oficjalnie: *"Warning raised when trying to set on a copied slice from a DataFrame. This can happen unintentionally when chained indexing."*[pandas.pydata.org](https://pandas.pydata.org/docs/reference/api/pandas.errors.SettingWithCopyWarning.html#:~:text=Warning%20raised%20when%20trying%20to,DataFrame). Innymi słowy, gdy robimy coś takiego jak **`df[some_condition]["col"] = value`**, istnieje ryzyko, że modyfikujemy obiekt, który nie jest oryginalnym **`df`**, tylko jego kawałkiem.




**Podsumowanie zaleceń:**

* Używaj **`.loc`** do modyfikacji części DataFrame: **`df.loc[mask, "col"] = new_value`**.

* Gdy otrzymasz SettingWithCopyWarning, przeanalizuj, czy nie robisz operacji na obiekcie, który jest wynikiem wycięcia danych. Zazwyczaj rozwiązaniem jest przebudowanie tej operacji na pojedyncze wywołanie **`.loc`** **albo** zastosowanie **`.copy()`** tam gdzie tworzysz podramkę.

* Można globalnie wyłączyć to ostrzeżenie, ale **nie jest to zalecane**, lepiej poprawić kod. W przyszłych wersjach Pandas (być może 3.0) planowane jest ulepszenie semantyki kopiowania (mechanizm Copy-on-Write), które może sprawić, że takie niejednoznaczności znikną, a ostrzeżenie stanie się niepotrzebne.




### **Typy kategoryczne i zarządzanie pamięcią**

Na koniec wspomnijmy o **typach kategorycznych** w pandas i ich wpływie na wydajność. Pandas oferuje typ **`Categorical`** dla zmiennych, które przyjmują ograniczony zbiór unikalnych wartości (kategorie). Przykłady: kolumna "day" w **`tips`** ma tylko kilka unikalnych wartości ("Thur","Fri","Sat","Sun"), kolumna "smoker" ma "Yes"/"No", kolumna "sex" ma "Male"/"Female". Zamiast przechowywać te wartości jako pełne stringi dla każdego wiersza, możemy je przekonwertować na typ kategorii, co wewnętrznie przechowuje liczbową reprezentację kategorii oraz słownik (tzw. *categories*) z mapowaniem na właściwe nazwy.

**Korzyści:**

* Oszczędność pamięci – zamiast powtarzać długi ciąg znaków wiele razy, przechowujemy jeden integer odwołujący się do kategorii.

* Często przyspieszenie operacji grupowania, porównywania – operacje na int są szybsze niż na stringach.

* Możliwość ustawienia porządku (order) kategorii, co bywa użyteczne przy sortowaniu czy wykresach.

**Jak użyć:**

Kolumny podczas wczytywania CSV można od razu zadeklarować jako kategorie (parametr **`dtype={"col": "category"}`** w read\_csv). Można też po wczytaniu przekonwertować:


In [ ]:
print("Typy przed:", tips.dtypes)
tips["day"] = tips["day"].astype("category")
tips["sex"] = tips["sex"].astype("category")
tips["smoker"] = tips["smoker"].astype("category")
tips["time"] = tips["time"].astype("category")
print("Typy po:", tips.dtypes)

Teraz np. **`tips["day"].dtype`** pokaże **`CategoricalDtype`**. Możemy zobaczyć jakie kategorie ma kolumna:


In [ ]:
print(tips["day"].cat.categories)





Powinno wypisać listę kategorii, np. **`Index(['Thur', 'Fri', 'Sat', 'Sun'], dtype='object')`**. Możemy też zmienić kolejność czy dodać/usunąć kategorie jeśli potrzeba (metody **`.cat.reorder_categories`**, **`.cat.add_categories`** itd.).

**Sprawdzenie zużycia pamięci:**

Pandas **`df.info(memory_usage="deep")`** pokaże szacowane zużycie pamięci kolumn. Można porównać przed i po konwersji na category, szczególnie dla dużych kolumn z powtarzającymi się wartościami:


In [ ]:

tips_small = pd.DataFrame({
    "city": ["Warsaw"] * 1000 + ["Paris"] * 1000 + ["London"] * 1000
})
print(tips_small.info(memory_usage="deep"))
tips_small["city_cat"] = tips_small["city"].astype("category")
print(tips_small.info(memory_usage="deep"))



Na 3000 wierszy z 3 unikalnymi miastami różnica będzie zauważalna – kolumna object (string) zużyje znacznie więcej pamięci niż kolumna category.
**Uwaga:** Nie wszystkie operacje świetnie działają z kategoriami (np. jeśli często dodajemy nowe kategorie dynamicznie, może być to upierdliwe, bo trzeba dodawać do listy kategorii). Jednak generalnie, dla kolumn typu "słownik" (np. kod kraju, płeć, kategoria produktu) warto używać kategorii, zwłaszcza na dużych DataFrame, aby oszczędzić pamięć.
To tyle jeśli chodzi o typy kategoryczne – traktuj to jako wskazówkę optymalizacyjną.


Powyżej omówiliśmy główne funkcjonalności pandas. Teraz czas na część praktyczną – kilka zadań, które pozwolą utrwalić te zagadnienia.



### **Zadania**

**Zadanie 6:** Utwórz **`DataFrame`** zawierający kolumny: "Name", "Age", "City" z dowolnymi danymi (minimum 5 wierszy). Następnie:

* Ustaw "Name" jako indeks wiersza (metodą **`set_index`**).

* Wypisz informacje o DataFrame (**`.info()`**).

* Przywróć domyślny indeks numeryczny (np. poprzez **`reset_index`**).

**Zadanie 7:** Korzystając z wczytanego wcześniej zbioru **`tips`** (jeśli nie wczytałeś, zrób to teraz), wykonaj następujące operacje:

* Wyfiltruj wiersze, gdzie **`time == 'Dinner'`** i **`tip > 5`**. Ile takich transakcji jest?

* Oblicz średnią wartość rachunku (**`total_bill`**) dla poszczególnych dni tygodnia.

* Dodaj nową kolumnę **`tip_pct`** (procent napiwku względem rachunku) do **`tips`**.

* Posortuj DataFrame **`tips`** malejąco po **`tip_pct`** i pokaż top 5 wierszy.


**Zadanie 8:** Stwórz dwa DataFrames:

* **`df1`** z kolumnami: "Product", "Price". Niech zawiera np. 4 produkty z cenami.

* **`df2`** z kolumnami: "Product", "Quantity". Niech zawiera informację o liczbie sprzedanych sztuk tych produktów (możesz dla uproszczenia powtórzyć te same produkty, albo pominąć/zmienić jeden by zobaczyć efekt braku dopasowania).

  Następnie:

* Połącz **`df1`** i **`df2`** w jeden DataFrame na kolumnie "Product", tak aby otrzymać tabelę z kolumnami: Product, Price, Quantity.

* Oblicz kolumnę **`TotalRevenue`** = **`Price * Quantity`**.

* (*Bonus:*) Jeśli w drugim DataFrame pominąłeś jeden produkt, spraw, by merge było typu "outer" i brakującą wartość potraktuj jako 0 sprzedanych sztuk (wypełnij NaN zerem).




**Zadanie 9:** Skorzystaj z DataFrame **`tips`** lub utwórz inny przykładowy DataFrame, aby wykonać operację pivot i melt:

* Użyj **`pivot_table`**, aby pokazać np. średnią kwotę napiwku (**`tip`**) dla kombinacji **`day`** (wiersze) i **`sex`** (kolumny).

* Przekształć wynik tej pivot\_table z powrotem do długiego formatu za pomocą **`melt`**.



**Zadanie 10:** Utwórz DataFrame zawierający pewne braki danych:

* Co najmniej 5 wierszy i 3 kolumny, w tym kilka wartości NaN/None.

  Następnie:

* Sprawdź ile brakuje wartości w każdej kolumnie.

* Usuń wiersze, które mają braki w **wszystkich** kolumnach.

* W pozostałych brakujące wartości wypełnij: dla kolumn numerycznych użyj średniej, dla tekstowych np. stringiem "unknown".


**Zadanie 11 (SettingWithCopyWarning):** Poniższy kod generuje ostrzeżenie SettingWithCopyWarning:




In [ ]:
df = pd.DataFrame({"A": [5, 7, 9, 4], "B": [1, 2, 3, 4]})
filtered = df[df["A"] > 5]
filtered["B"] = 0

* Uruchom ten kod i zaobserwuj ostrzeżenie.

* Popraw kod tak, aby nie generował ostrzeżenia, a zmiana wartości w kolumnie B była poprawnie zastosowana:

  * **Wariant a)** modyfikując oryginalny **`df`**,

  * **Wariant b)** na kopii (**`filtered`**), ale wtedy nie zmieniaj oryginału (dodaj **`.copy()`**).

* Wyjaśnij w komentarzu, która metoda modyfikacji danych jest preferowana w pandas i dlaczego.
